In [1]:
import pandas as pd



In [2]:
df = pd.read_csv('../data/processed/cleaned_data.csv')

In [3]:
df

,handle_count,no_handle_no_special_no_sin_ash,character_count,lemmatized,tweet_without_stopwords,tweet_without_stopwords2,pos_label
0,0,draymond green a bitch ass nigga bruh,31,draymond green a bitch as nigga bruh,draymond green bitch nigga bruh,draymond green bitch nigga bruh,0
1,0,i decided to let you go after all #reality,34,i decide to let you go after all # reality,decide let go # reality,decide let reality,1
2,0,another shooting after grimmie s incident ...,107,another shooting after grimmie s incident dead...,another shooting grimmie incident dead orlando...,another shooting grimmie incident dead orlando...,1
3,0,baptize my great nephew max tomorrow,31,baptize my great nephew max tomorrow,baptize great nephew max tomorrow,baptize great nephew max tomorrow,1
4,1,RT First off fuck your bitch And the click...,42,RT First off fuck your bitch And the click you...,RT First fuck bitch And click claim,First fuck bitch And click claim,0
...,...,...,...,...,...,...,...
47778,0,I m going to sell my iPhone and both my Macs ...,53,I m go to sell my iPhone and both my Macs I do...,I go sell iPhone Macs I support faggot,sell iPhone Macs support faggot,0
47779,0,hello darkness my old friend #scotland #sun...,62,hello darkness my old friend # scotland # suns...,hello darkness old friend # scotland # sunset ...,hello darkness old friend scotland sunset hill...,1
47780,0,i can t believe that my antidepressants are wo...,79,i can t believe that my antidepressant be work...,believe antidepressant work know go work,believe antidepressant work know work,1
47781,0,#porn #android #iphone #ipad #sex #xxx #Ana...,70,# porn # android # iphone # ipad # sex # xxx #...,# porn # android # iphone # ipad # sex # xxx #...,porn android iphone ipad sex xxx Anal Hardcore...,0
